In [1]:
from contextlib import contextmanager
import os
import datetime as dt
import numpy as np
import h5py
import time
import sys
#os.chdir('../telescope_control/data_aquisition/')
sys.path.append('../')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_meinhold/')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_zonca/')
from glob import glob

sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('..//VtoT//')

import Tkinter,tkFileDialog

import cofe_util as cu
import realtime_gp as rt
import datetime

from Tkinter import *
import ttk

In [2]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [3]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
def plot_some_chans(plottype='toi',filelist=None,component='T',samprate=30,minfreq=.1):
    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC H1HiDC H1LoAC H1LoDC H2HiAC H2HiDC H2LoAC H2LoDC\
                 H3HiAC H3HiDC H3LoAC H3LoDC HornTop Amplifier Cooler Transition")           
    dlist=[]
    tlist=[]
    for f in filelist[1:]:
        h=np.float64(f[-11:-9])
        m=np.float64(f[-9:-7])
        s=np.float64(f[-7:-5])
        hf=h5py.File(f)
        t=h+m/60.+(s+(hf['demod_data']['rev']-hf['demod_data']['rev'][0])/1000.)/3600.
        tlist.append(t)  #just generate a time to plot relative to
        dlist.append(hf['demod_data'])
    d=concatenate(dlist)
    ut=concatenate(tlist)
    hf.close() 
    
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        title('%s TOI from %s %s %s' %(component,mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val)
            plot(ut,d[chan][component],label=val)
            xlabel('UT [h]')
            ylabel('output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        title('%s ASD from %s %s %s' %(component,mm,dd,yyyy))

        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val)
            freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
            plot(freq,sqrt(psd)*1e9,label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{nV}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="Select for TOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="Select for ASD", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [5]:
def plot_some_chans2(datatype='demod',plottype='toi',filelist=None,component='T',samprate=30,minfreq=.1):
    
    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC H1HiDC H1LoAC H1LoDC H2HiAC H2HiDC H2LoAC H2LoDC\
                 H3HiAC H3HiDC H3LoAC H3LoDC HornTop Amplifier Cooler Transition")           
    tlist=[]
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            h=np.float64(f[-11:-9])
            m=np.float64(f[-9:-7])
            s=np.float64(f[-7:-5])
            hf=h5py.File(f)
            t=h+m/60.+(s+(hf['demod_data']['rev']-hf['demod_data']['rev'][0])/1000.)/3600.
            tlist.append(t)  #just generate a time to plot relative to
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        h=np.float64(filelist[0][-11:-9])
        m=np.float64(filelist[0][-9:-7])
        s=np.float64(filelist[0][-7:-5])
        d=rt.demod.read_raw(filelist)
        t=h+m/60.+(s+(d['rev']-d['rev'][0])/1000.)/3600.
        tlist.append(t)
    ut=concatenate(tlist)
    #figure()
    #plot(ut)
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        if datatype=='demod':
            title('%s TOI from %s %s %s' %(component,mm,dd,yyyy))
        if datatype=='raw':
            title('TOI from %s %s %s' %(mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val)
            if datatype=='demod':
                plot(ut,d[chan][component],label=val)
            if datatype=='raw':
                plot(d[chan].flatten(),label=val)
            xlabel('Samples')
            ylabel('Output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        if datatype=='demod':
            title('%s ASD from %s %s %s' %(component,mm,dd,yyyy))
        if datatype=='raw':
            title('ASD from %s %s %s' %(mm,dd,yyyy))
                

        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val)
            if datatype=='demod':
                freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
            plot(freq,sqrt(psd),label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="Select for TOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="Select for ASD", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [8]:
plot_some_chans2(datatype='raw',minfreq=.1)

1000
16


In [ ]:
def getBP(DClevel, WN):
    